<a href="https://colab.research.google.com/github/dariashcherbakovaaa/MA_DSE_Unimi/blob/Thesis/Transfromers_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 18.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 18.0.0 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/to_sent/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import warnings
warnings.filterwarnings("ignore")

# library loadings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import data_preprocessing
from data_preprocessing import *
import my_func
from my_func import *

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

import torchvision
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau

import datasets
import transformers
from transformers import AutoModel, AutoTokenizer, BertTokenizer, BertModel, BertConfig, XLMRobertaTokenizer

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# data loading


In [ ]:
df1 = pd.read_excel('/content/drive/MyDrive/to_sent/Verbatim_Maxidi - Mattia.xlsx').rename(columns={'D15' : 'text'}).drop(columns=['Unnamed: 18'])
df2 = pd.read_excel('/content/drive/MyDrive/to_sent/Verbatim_Suggerimenti_Superemme.xlsx').rename(columns={'Verbatim': 'text',
                                                                                 'Ricodifica1': 'Cod1', 'Ricodifica2': 'Cod2',
                                                                                 'Ricodifica3': 'Cod3','Ricodifica4': 'Cod4',
                                                                                 'Ricodifica5': 'Cod5', 'Ricodifica6': 'Cod6',
                                                                                 'Ricodifica7': 'Cod7', 'Ricodifica8': 'Cod8'})
df3 = pd.read_excel('/content/drive/MyDrive/to_sent/Cds_verbatim.xlsx').rename(columns={'D15':'text'})

df1 = df1.iloc[: , 1:]
df2 = df2.iloc[: , 1:]
df3 = df3.iloc[:,1:]

database = pd.concat([df1, df2, df3], ignore_index=True)

In [ ]:
database['all_codes'] = database.iloc[:, 1:].apply(lambda x: '; '.join(x.dropna()), axis=1)
data = database[['text','all_codes']].copy()
data.columns = ['text', 'label']

data = data.drop_duplicates()
data.dropna(inplace = True)
print(f"The total database size: {data.shape}")
print(f"# unique answers: {data.text.nunique()}")
print(f"# unique labels: {data.label.nunique()}")

data.head()

The total database size: (2481, 2)
# unique answers: 2479
# unique labels: 740


,text,label
0,personale di servizio barriera casse,Casse; Personale
1,nella pulizia e nella professionalità dei resp...,Promozioni; Ordine&Pulizia; Personale
2,più varietà e scaffali ben riforniti quando ci...,Promozioni; Riapprovigionamento; Varietà
3,sicuramente la cortesia alle casse!!,Casse; Personale
4,prodotti freschi senza glutine,Gluten Free


In [ ]:
labels = data.label.str.split('; ', expand=True)
data = pd.concat([data[['text']], labels], axis=1)
df = db_unite(data)
df.head(4)

,text,label
0,personale di servizio barriera casse,Casse
1,personale di servizio barriera casse,Personale
2,nella pulizia e nella professionalità dei resp...,Promozioni
3,nella pulizia e nella professionalità dei resp...,Ordine&Pulizia


In [ ]:
df.label = df.label.apply(lambda x: x.strip().lower())
df.head(7)

,text,label
0,personale di servizio barriera casse,casse
1,personale di servizio barriera casse,personale
2,nella pulizia e nella professionalità dei resp...,promozioni
3,nella pulizia e nella professionalità dei resp...,ordine&pulizia
4,nella pulizia e nella professionalità dei resp...,personale
5,più varietà e scaffali ben riforniti quando ci...,promozioni
6,più varietà e scaffali ben riforniti quando ci...,riapprovigionamento


In [ ]:
recoding = {
            "esposizione pdt" : 'esposizione prodotto',
            'bio': 'bio, gluten free, vegan', 'gluten free': 'bio, gluten free, vegan', 'vegan': 'bio, gluten free, vegan', 'no food': 'nofood',
            'scadenze pdt': 'controllo scadenze'
            }
df['label'] = df['label'].replace(recoding).str.lower().str.strip()

In [ ]:
list_labels = df.label[df.label.map(df.label.value_counts() < 30)].unique()
print(df.label.nunique())
print(df.label.value_counts())

43
label
assortimento               529
prezzi                     435
promozioni                 390
personale                  361
casse                      252
ortofrutta                 246
riapprovigionamento        240
qualità                    188
ordine&pulizia             188
gastronomia                158
macelleria                 142
pescheria                  132
varietà                    118
freschezza                 111
localismi                   93
nulla                       92
bio, gluten free, vegan     85
panetteria                  73
chiarezza prezzi            71
esposizione prodotto        68
parcheggio                  67
marche                      58
controllo scadenze          47
zzz                         47
carta fedeltà               46
volantini                   37
spesa online                35
surgelati                   32
drug                        31
buoni pasto                 28
carrelli&cestini            23
petfood                     18

In [ ]:

idx_drop = df[df.label == 'zzz'].index
df.drop(idx_drop, inplace=True)

In [ ]:
database = df.groupby('text')['label'].apply(list).reset_index()
print(database.shape)

(2432, 2)


In [ ]:
for idx, lists in enumerate(database['label']):
    if any(elem in list_labels for elem in lists):
        database.drop(idx, inplace=True)
database.reset_index(drop=True, inplace=True)
database.shape

(2308, 2)

In [ ]:
def dataframe_creation(df):
    mlb = MultiLabelBinarizer()
    label_binarized = mlb.fit_transform(df['label'])
    df_coded = pd.DataFrame(label_binarized, columns=mlb.classes_, index=df['text'])
    df_coded.reset_index(inplace=True)
    df_coded['tag_list'] = df_coded[df_coded.columns[1:]].values.tolist()
    new_df = df_coded[['text', 'tag_list']].copy()
    new_df['label_names'] = new_df['tag_list'].apply(lambda tags: [mlb.classes_[i] for i, tag in enumerate(tags) if tag == 1])
    #new_df['label_names'] = new_df['label_names'].apply(lambda x: ", ".join(map(str, x)))
    return new_df,mlb.classes_

df, label_names = dataframe_creation(database)
df['tag_list'] = df['tag_list'].apply(lambda x: [int(i) for i in x])
df['label_count'] = df['tag_list'].apply(lambda x: x.count(1))
df.dropna(inplace = True)
df

,text,tag_list,label_names,label_count
0,Copertura Parcheggio,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[parcheggio],1
1,"Velocità nelle casse , aiuto in salumeria","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[casse],1
2,buoni spesa piu' consistenti.visto il costo ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[prezzi],1
3,i volantini con le offerte non sono così faci...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[promozioni, volantini]",2
4,il reparto ortofrutta dovrebbe essere riforni...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[ortofrutta, riapprovigionamento]",2
...,...,...,...,...
2303,è un bellissimo supermercato e super fornito m...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[prezzi],1
2304,è un insegna che si vede va bene così,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",[nulla],1
2305,"è un ottimo punto vendita,ma spesso i prezzi n...","[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[chiarezza prezzi, esposizione prodotto, riapp...",3
2306,è un po' trascurato: a volte dell'acqua piove ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",[ordine&pulizia],1


In [ ]:
df['word_count'] = df.text.astype(str).apply(word_count)

print(f"min lenght: {np.min(df['word_count'])}")
print(f"max lenght: {np.max(df['word_count'])}")
print(f"avg lenght: {np.mean(df['word_count'])}")
print(np.percentile(df['word_count'], [95, 99]))

min lenght: 1
max lenght: 294
avg lenght: 13.43370883882149
[40.   87.86]


# Transfromers setting

In [ ]:
MAX_LEN = 100
TRAIN_BATCH_SIZE = 6
VALID_BATCH_SIZE = 4
EPOCHS = 10
LEARNING_RATE = 2e-05

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }


test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

In [ ]:
class CustomDataset(Dataset):
  def __init__(self, dataframe, tokenizer, max_len):
    self.data = dataframe
    self.comment_text = dataframe.text
    self.targets = self.data.tag_list
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self): return len(self.comment_text)

  def __getitem__(self, index):
    comment_text = str(self.comment_text[index])
    comment_text = " ".join(comment_text.split())

    inputs = self.tokenizer.encode_plus(
        comment_text,
        None,
        add_special_tokens=True,
        max_length=self.max_len,
        pad_to_max_length=True,
        return_token_type_ids=True,
        truncation=True)

    ids = inputs['input_ids']
    mask = inputs['attention_mask']
    token_type_ids = inputs["token_type_ids"]


    return {
        'ids': torch.tensor(ids, dtype=torch.long),
        'mask': torch.tensor(mask, dtype=torch.long),
        'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
        'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
class Classifier(torch.nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.l1 = transformers.AutoModel.from_pretrained(model_checkpoint)
        self.l2 = torch.nn.Dropout(0)
        self.l3 = torch.nn.Linear(768, 28)

    def forward(self, ids, mask, token_type_ids):
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        return output

In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [ ]:
def train(epoch):
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)
        outputs = model(ids, mask, token_type_ids)

        loss = loss_fn(outputs, targets)
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)

            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

# Weighted

# Stratifighed

In [ ]:
! pip install iterative-stratification

In [ ]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import numpy as np

# Prepare your labels as a binary matrix for stratification
y = np.array(df['tag_list'].tolist())

# Define the stratified KFold object
mskf = MultilabelStratifiedKFold(n_splits=5, random_state=200, shuffle=True)

# Get train/test indices
train_indices, test_indices = next(mskf.split(df['text'], y))

# Use these indices to create train and test sets
train_dataset = df.iloc[train_indices].reset_index(drop=True)
test_dataset = df.iloc[test_indices].reset_index(drop=True)

print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

FULL Dataset: (2308, 5)
TRAIN Dataset: (1848, 5)
TEST Dataset: (460, 5)


In [ ]:
model_checkpoint = 'dbmdz/bert-base-italian-cased'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, padding='max_length', truncation=True, return_tensors = 'pt')

tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/235k [00:00<?, ?B/s]

In [ ]:
training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN)

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
model = Classifier()
model.to(device)

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

Classifier(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31102, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affi

In [ ]:
optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE, weight_decay=0)

In [ ]:
for epoch in range(EPOCHS):
    train(epoch)

Epoch: 0, Loss:  0.6914322972297668
Epoch: 1, Loss:  0.3015279173851013
